In [2]:
import numpy as np
import pandas as pd
from preprocess_code import convert_to_graph2vec

In [ ]:
import swifter

In [3]:
data = pd.read_csv('../data/juliet.csv.zip')

In [ ]:
graph2vec = data.apply(convert_to_graph2vec, axis = 'columns')

In [ ]:
list(map(convert_to_graph2vec,data.apply))

In [5]:
data.shape[0]

66347

In [6]:
num_of_data = data.shape[0]

In [7]:
small_data = data.iloc[:num_of_data,:]

In [9]:
import clang.cindex
import os
clang.cindex.Config.set_library_file('/lib/x86_64-linux-gnu/libclang-8.so.1')

In [ ]:
def get_trainslation_units(i):
    try:
        index_clang = clang.cindex.Index.create()
        translation_unit = []
        value = small_data.iloc[i]
        translation_unit = index_clang.parse(path=value.filename, unsaved_files=[(value.filename, value.code)])

        return translation_unit
    
    except:
        import pdb; pdb.set_trace()
        return None

%time 
translation_unit = list(map(get_trainslation_units,range(num_of_data)))


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.7 µs


In [ ]:
# start at 11.41

In [ ]:
translation_unit = []
for i in range(num_of_data):
    value = small_data.iloc[i]
    translation_unit.append(index_clang.parse(path=value.filename, unsaved_files=[(value.filename, value.code)]))
    
translation_unit

In [ ]:
roots = []
for i in range(num_of_data):
    roots.append(translation_unit[i].cursor)

In [86]:
import snap

In [87]:
def get_identifier(root):
    
    def number_each_node(node, counter=1):
        node.identifier = counter
        counter += 1

        node.children = list(node.get_children())
        for child in node.children:
            counter = number_each_node(child, counter)
            
        return counter
        
    number_each_node(root)
    return root

roots = list(map(get_identifier,roots))

In [88]:
def print_ast(node):
    print(str(node.kind) + " " + str(node.identifier))
    for child in node.children:
        print_ast(child)

In [89]:
print_ast(roots[0])

CursorKind.TRANSLATION_UNIT 1
CursorKind.TYPEDEF_DECL 2
CursorKind.TYPEDEF_DECL 3
CursorKind.TYPEDEF_DECL 4
CursorKind.TYPEDEF_DECL 5
CursorKind.TYPEDEF_DECL 6
CursorKind.STRUCT_DECL 7
CursorKind.FIELD_DECL 8
CursorKind.UNION_DECL 9
CursorKind.FIELD_DECL 10
CursorKind.FIELD_DECL 11
CursorKind.INTEGER_LITERAL 12
CursorKind.FIELD_DECL 13
CursorKind.UNION_DECL 14
CursorKind.FIELD_DECL 15
CursorKind.FIELD_DECL 16
CursorKind.INTEGER_LITERAL 17
CursorKind.TYPEDEF_DECL 18
CursorKind.STRUCT_DECL 19
CursorKind.FIELD_DECL 20
CursorKind.UNION_DECL 21
CursorKind.FIELD_DECL 22
CursorKind.FIELD_DECL 23
CursorKind.INTEGER_LITERAL 24
CursorKind.FIELD_DECL 25
CursorKind.UNION_DECL 26
CursorKind.FIELD_DECL 27
CursorKind.FIELD_DECL 28
CursorKind.INTEGER_LITERAL 29
CursorKind.TYPEDEF_DECL 30
CursorKind.TYPE_REF 31
CursorKind.STRUCT_DECL 32
CursorKind.TYPEDEF_DECL 33
CursorKind.TYPE_REF 34
CursorKind.STRUCT_DECL 35
CursorKind.TYPEDEF_DECL 36
CursorKind.TYPE_REF 37
CursorKind.STRUCT_DECL 38
CursorKind.STRUC

CursorKind.PARM_DECL 2178
CursorKind.PARM_DECL 2179
CursorKind.PARM_DECL 2180
CursorKind.FUNCTION_DECL 2181
CursorKind.UNEXPOSED_ATTR 2182
CursorKind.UNEXPOSED_ATTR 2183
CursorKind.PARM_DECL 2184
CursorKind.FUNCTION_DECL 2185
CursorKind.UNEXPOSED_ATTR 2186
CursorKind.UNEXPOSED_ATTR 2187
CursorKind.PARM_DECL 2188
CursorKind.PARM_DECL 2189
CursorKind.PARM_DECL 2190
CursorKind.FUNCTION_DECL 2191
CursorKind.UNEXPOSED_ATTR 2192
CursorKind.UNEXPOSED_ATTR 2193
CursorKind.PARM_DECL 2194
CursorKind.FUNCTION_DECL 2195
CursorKind.UNEXPOSED_ATTR 2196
CursorKind.TYPE_REF 2197
CursorKind.PARM_DECL 2198
CursorKind.FUNCTION_DECL 2199
CursorKind.UNEXPOSED_ATTR 2200
CursorKind.PARM_DECL 2201
CursorKind.PARM_DECL 2202
CursorKind.TYPE_REF 2203
CursorKind.FUNCTION_DECL 2204
CursorKind.FUNCTION_DECL 2205
CursorKind.UNEXPOSED_ATTR 2206
CursorKind.PARM_DECL 2207
CursorKind.PARM_DECL 2208
CursorKind.FUNCTION_DECL 2209
CursorKind.UNEXPOSED_ATTR 2210
CursorKind.UNEXPOSED_ATTR 2211
CursorKind.PARM_DECL 2212
Curso

In [90]:
tree_all = [snap.TNGraph.New()]*10

def tree2edgelist(node, tree = snap.TNGraph.New()):
    tree.AddNode(node.identifier)

    for child in node.children:
        tree2edgelist(child, tree)
        tree.AddEdge(node.identifier, child.identifier)
    return tree


In [91]:
def get_all_tree(root):
    tree = snap.TNGraph.New()

    def tree2edgelist(node):
        tree.AddNode(node.identifier)

        for child in node.children:
            tree2edgelist(child)
            tree.AddEdge(node.identifier, child.identifier)
    tree2edgelist(root)
    return tree

In [92]:
trees = list(map(get_all_tree,roots))

In [93]:
trees

[<snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882e70> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882cf0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9bb8cab270> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882c00> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882b10> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882d80> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b98882de0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b8bbfe090> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b8bbfe120> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b8bbfe1b0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b8bbfe240> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f9b8bbfe2d0> >,
 <snap.PNGraph; proxy of <Swig Object of

In [53]:
def deg_node(node):
    deg = GetInDeg(node) + GetOutDeg(node)
    return deg

In [67]:
def get_deg(tree):
    deg_node = {}
    for node in tree.Nodes():
        deg_node[str(node.GetId())] = str(node.GetInDeg() + node.GetOutDeg())
    return deg_node
list_deg_nodes = list(map(get_deg,trees))

In [63]:
def get_edges(tree):
    edges = []

    for edge in tree.Edges():
        edges.append([edge.GetSrcNId(), edge.GetDstNId()])
    return edges
list_edges = list(map(get_edges,trees))

In [66]:
deg_nodes
list_edges

[[[1, 2],
  [1, 3],
  [1, 4],
  [1, 5],
  [1, 6],
  [1, 7],
  [1, 18],
  [1, 30],
  [1, 32],
  [1, 33],
  [1, 35],
  [1, 36],
  [1, 38],
  [1, 48],
  [1, 50],
  [1, 52],
  [1, 53],
  [1, 56],
  [1, 59],
  [1, 62],
  [1, 65],
  [1, 71],
  [1, 78],
  [1, 82],
  [1, 87],
  [1, 93],
  [1, 100],
  [1, 104],
  [1, 106],
  [1, 112],
  [1, 114],
  [1, 117],
  [1, 120],
  [1, 123],
  [1, 126],
  [1, 129],
  [1, 132],
  [1, 135],
  [1, 137],
  [1, 140],
  [1, 143],
  [1, 146],
  [1, 152],
  [1, 154],
  [1, 156],
  [1, 158],
  [1, 162],
  [1, 166],
  [1, 171],
  [1, 173],
  [1, 175],
  [1, 177],
  [1, 179],
  [1, 181],
  [1, 183],
  [1, 185],
  [1, 187],
  [1, 191],
  [1, 195],
  [1, 199],
  [1, 204],
  [1, 209],
  [1, 214],
  [1, 219],
  [1, 221],
  [1, 223],
  [1, 228],
  [1, 233],
  [1, 237],
  [1, 239],
  [1, 244],
  [1, 249],
  [1, 252],
  [1, 258],
  [1, 262],
  [1, 264],
  [1, 268],
  [1, 273],
  [1, 276],
  [1, 281],
  [1, 286],
  [1, 289],
  [1, 294],
  [1, 300],
  [1, 304],
  [1, 310],


In [70]:
def get_json_files(edges,deg_node):
    graph2vec_example = {
        "edges" : edges,
        "features" : deg_node
    }
    return graph2vec_example
json_files = list(map(get_json_files,list_edges,list_deg_nodes))

In [72]:
import json
for i in range(num_of_data):
    with open('../data/graph2vec_examples/' + str(i) + '.json', 'w') as f:
        json.dump(json_files[i], f)

In [74]:
cat ../data/graph2vec_examples/1.json

{"edges": [[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 18], [1, 30], [1, 32], [1, 33], [1, 35], [1, 36], [1, 38], [1, 48], [1, 50], [1, 52], [1, 53], [1, 56], [1, 59], [1, 62], [1, 65], [1, 71], [1, 78], [1, 82], [1, 87], [1, 93], [1, 100], [1, 104], [1, 106], [1, 112], [1, 114], [1, 117], [1, 120], [1, 123], [1, 126], [1, 129], [1, 132], [1, 135], [1, 137], [1, 140], [1, 143], [1, 146], [1, 152], [1, 154], [1, 156], [1, 158], [1, 162], [1, 166], [1, 171], [1, 173], [1, 175], [1, 177], [1, 179], [1, 181], [1, 183], [1, 185], [1, 187], [1, 191], [1, 195], [1, 199], [1, 204], [1, 209], [1, 214], [1, 219], [1, 221], [1, 223], [1, 228], [1, 233], [1, 237], [1, 239], [1, 244], [1, 249], [1, 252], [1, 258], [1, 262], [1, 264], [1, 268], [1, 273], [1, 276], [1, 281], [1, 286], [1, 289], [1, 294], [1, 300], [1, 304], [1, 310], [1, 314], [1, 318], [1, 320], [1, 325], [1, 330], [1, 333], [1, 334], [1, 338], [1, 344], [1, 346], [1, 347], [1, 348], [1, 349], [1, 350], [1, 351], [1, 352], [